In [15]:
pip install geopandas rioxarray rasterio xarray

In [16]:
pip install earthengine-api


Note: you may need to restart the kernel to use updated packages.


In [17]:
!pip install geemap
!pip install geopandas
!pip install fiona

In [4]:
# Importar las librerías necesarias
import ee
import geemap
import geopandas as gpd

# Autenticación para Jupyter Lab (esto generará un enlace dentro del notebook)
ee.Authenticate(auth_mode='notebook')

# Inicializar Earth Engine (si tienes un proyecto GEE, puedes incluir el ID aquí)
ee.Initialize(project='ee-maperezla')


In [5]:
import geopandas as gpd

# Cargar shapefile
shapefile_path = 'C:/Users/Equipo/SemDocker/ProyectoProgramacionSIG/AOI/PNNAmacayacu.shp'  # Modifica esta ruta
gdf = gpd.read_file(shapefile_path)

# Convertir a formato GeoJSON y luego a ee.Geometry
import json
from shapely.geometry import mapping

geometry = ee.Geometry(mapping(gdf.geometry[0]))  # Usa la primera geometría del shapefile


In [6]:
bandas = ['B2', 'B3', 'B4', 'B5','B6','B8','B8A','B9','B11','B12']
sentinel2 = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2023-11-01', '2023-12-31')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .filterBounds(geometry)
    .select(bandas)
    .median()
    .clip(geometry)
)

In [7]:
export_task = ee.batch.Export.image.toDrive(
    image=sentinel2,
    description='sentinel2_25_06_11',
    folder='Imagenes_GEE',  # Asegúrate de que existe
    fileNamePrefix='sentinel2_25_06_11',
    region=geometry.bounds().getInfo()['coordinates'],
    scale=10,
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)
export_task.start()